<a href="https://colab.research.google.com/github/chkla/codecarbon-huggingface/blob/main/CodeCarbonCometHuggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing CodeCarbon 💨 with Huggingface 🤗

In this notebook, I played around with the new [CodeCarbon](https://codecarbon.io/) 💨 package integrated into [Comet](https://www.comet.ml/) ☄️ using [Huggingface](https://huggingface.co/) 🤗 to show a fine-tuned language model's carbon footprint.

In 2019 the paper ["Energy and Policy Considerations for Deep Learning in NLP"](https://arxiv.org/pdf/1906.02243.pdf) popped up, discussing machine learning models' carbon footprint. Giving this as a portion of food for thought, the community starts thinking about the long-term effects and consequences.

The [CodeCarbon](https://codecarbon.io/) 💨 project is a software package to track the carbon footprint. This package is already integrated into [Comet](https://www.comet.ml/) ☄️ , a tool to analyze and track your models (similar to [wandb](https://wandb.ai/)).

To exemplify the use of CodeCarbon 💨, I used a part of code from [this](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb#scrollTo=uNx5pyRlIrJh)and  HuggingFace' notebook to define a simple task for fine-tuning a language model (if you want, you can try out any other task).

_Note: The current integration in HuggingFace seems to be a bit buggy in logging the experiments in the right format to get a carbon score._

In [ ]:
%pip install comet_ml>=3.2.2
%pip install transformers==4.2.0
%pip install datasets
%pip install codecarbon

In [1]:
from comet_ml import Experiment

In [2]:
import comet_ml

experiment = comet_ml.Experiment("api-key")
COMET_MODE = "online"
comet_ml.config.save(api_key="api-key") # without using this it the key is missing during the training

In [3]:
from datasets import load_dataset, load_metric
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [4]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
dataset = load_dataset("glue", "mnli")
metric = load_metric('glue', "mnli")

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [9]:
def preprocess_function(examples):
    if "hypothesis" is None:
        return tokenizer(examples["premise"], truncation=True)
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

In [12]:
args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation_matched"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [15]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.train()

In [ ]:
experiment = comet_ml.Experiment("api-key")
COMET_MODE = "online"